In [1]:
import numpy as np # Importo algebra Lineal
import matplotlib.pyplot as plt # Importo librería de gráficos 
import copy
import gmsh #Para hacer el mallado

In [2]:
#Cargo los datos del problema

cp=0.3 #Coeficiente de poisson
E=30E6 #[Psi]
t=1 # espesor [inch] 
largo=20 #[inch]
altura=10 #[inch]
T=1000 #[PSI]


In [3]:
gmsh.initialize() #Abre el programa en phyton

In [4]:
gmsh.model.add('Agujero') #Armamos el modelo que vamos modificar

In [5]:

lc=1 #Factor de escala

#La idea es resolver 1/4 de la chapa porque el problema es simetrico
p1 = gmsh.model.geo.addPoint(0, 0, 0,lc)
p2 = gmsh.model.geo.addPoint(0, altura/2, 0,lc)
p3 = gmsh.model.geo.addPoint(largo/2, altura/2, 0,lc) 
p4 = gmsh.model.geo.addPoint(largo/2, 0, 0,lc)
p5 = gmsh.model.geo.addPoint(0,1,0,lc/2)
p6 = gmsh.model.geo.addPoint(1,0,0,lc/2)

In [6]:
# Hay que definir todo de forma jerargica
#Con esto creo las lineas que componen la geometria

l1 = gmsh.model.geo.addLine(p5, p2)
l2 = gmsh.model.geo.addLine(p2, p3)
l3 = gmsh.model.geo.addLine(p3, p4)
l4 = gmsh.model.geo.addLine(p4, p6)
l5= gmsh.model.geo.addCircleArc(p5,p1 ,p6)  #Creo una parte circular


C1 = gmsh.model.geo.addCurveLoop([l1, l2, l3, l4])
C2 = gmsh.model.geo.addCurveLoop([l5])

#Definos la superficie usando el borde

S1 = gmsh.model.geo.addPlaneSurface([C1,C2]) #asi se genera la superifice descontando el circulo

In [7]:
gmsh.model.geo.synchronize() #Vuelve a sincronizar

In [8]:
#Physical groups

Empotradoy=gmsh.model.addPhysicalGroup(1,[l4])
gmsh.model.setPhysicalName(1,Empotradoy,'Empotradoy')

Traccionado=gmsh.model.addPhysicalGroup(1,[l3])
gmsh.model.setPhysicalName(1,Traccionado,'Traccionado')

Empotradox=gmsh.model.addPhysicalGroup(1,[l1])
gmsh.model.setPhysicalName(1,Empotradox,'Empotrado x')

Superficie = gmsh.model.addPhysicalGroup(2,[S1])
gmsh.model.setPhysicalName(2,Superficie, 'Superficie')

NodoAgujero= gmsh.model.addPhysicalGroup(0, [p1])
gmsh.model.setPhysicalName(0, NodoAgujero, 'Nodo agujero')


In [9]:
gmsh.model.mesh.generate(2)
gmsh.fltk.run()

In [10]:
NodeInfo = gmsh.model.mesh.get_nodes()
NumeroNodos=NodeInfo[0].shape[0]
MN=NodeInfo[1].reshape(NumeroNodos,3)

In [11]:
ETAGS,ELEMENTS=gmsh.model.mesh.get_elements_by_type(2)
MC=ELEMENTS.reshape([ETAGS.shape[0],3])
MC=(MC-1).astype(int)
print('La cantidad de elementos es',len(MC))


La cantidad de elementos es 60


In [12]:

NodosEmpotrados=gmsh.model.mesh.get_nodes_for_physical_group(1,Empotradox)
NodosEmpotrados2=gmsh.model.mesh.get_nodes_for_physical_group(1,Empotradoy)

s1=NodosEmpotrados[0].reshape(len(NodosEmpotrados[0]),1) # etiquetas de los nodos empotrados
s2=NodosEmpotrados2[0].reshape(len(NodosEmpotrados2[0]),1) #etiquetas de los nodos empotrados 2 
S=np.vstack((s1,s2))
z=np.zeros([len(NodosEmpotrados[0])+len(NodosEmpotrados2[0]),2])
MNE=np.concatenate((S-1,z),axis=1) # indices python
MNE=MNE.astype(int) 

NodosTraccionados=gmsh.model.mesh.get_nodes_for_physical_group(1,Traccionado)

s=NodosTraccionados[0].reshape(len(NodosTraccionados[0]),1).astype(int) # etiquetas de los nodos traccionados
z=np.zeros([len(NodosTraccionados[0]),2]).astype(int)
MNT=np.concatenate((s-1,z),axis=1) # indices python




In [13]:
#El desplazamiento nulo  a: Borde izquierdo (solo en x), Borde inferior (solo en y) ,Punto del medio de la chapa (en x e y)

agujero1 = 2 * (gmsh.model.mesh.get_nodes_for_physical_group(0,NodoAgujero)[0] - 1).astype(int)
agujero2= (2 * (gmsh.model.mesh.get_nodes_for_physical_group(0,NodoAgujero)[0] - 1) + 1).astype(int)
Smx = 2 * (gmsh.model.mesh.get_nodes_for_physical_group(1,Empotradox)[0] - 1).astype(int)
Smy = (2 * (gmsh.model.mesh.get_nodes_for_physical_group(1,Empotradoy)[0] - 1) +1).astype(int)


In [14]:

entityTraccionada=gmsh.model.getEntitiesForPhysicalGroup(1,Traccionado)
Tgroup,Ttraccionada,Ltraccionada=gmsh.model.mesh.getElements(1,entityTraccionada[0])
Ltraccionada=Ltraccionada[0].reshape(Ttraccionada[0].shape[0],2) #es la matriz de conectividad del extremo traccionado

#Para distribuir la tension en los elementos de la linea derecha

Longitudes=np.abs(MN[Ltraccionada[:,0]-1,1]-MN[Ltraccionada[:,1]-1,1])
Tension_ext=T/(altura)

#Calculo de las fuerzas sobre la linea derecha

for l,linea in enumerate(Ltraccionada-1):
    n1 = int(linea[0])   # el -1 es para numerar desde cero
    n2 = int(linea[1])
    flocal=Tension_ext*Longitudes[l]*t/2
    flocal=round(flocal)
    MNT[MNT[:,0]==n1,1] += flocal
    MNT[MNT[:,0]==n2,1] += flocal
    

print(MNT)

[[  2  83   0]
 [  3  83   0]
 [ 12 166   0]
 [ 13 166   0]]


In [15]:
#ACA EMPIEZAN LOS CALCULOS

nb=len(MNE)
nc=len(MNT)
n=len(MN)
print('el numero de nodos es',n)

D=np.array([[1,cp,0],
            [cp,1,0],
            [0,0,0.5*(1-cp)]] )*E/(1-cp**2)



el numero de nodos es 42


In [16]:
#Matriz de rigidez k global

kglobal=np.zeros([n*2,n*2])
B=[]
for i in range (len(MC)):
    A=np.linalg.det([[1,MN[MC[i,0],0],MN[MC[i,0],1]],
                    [1,MN[MC[i,2],0],MN[MC[i,2],1]],
                    [1,MN[MC[i,1],0],MN[MC[i,1],1]]])/2
    
    
    betai=MN[MC[i,2],1]-MN[MC[i,1],1]
    gammai=MN[MC[i,1],0]-MN[MC[i,2],0]
    
    betaj=MN[MC[i,1],1]-MN[MC[i,0],1]
    gammaj=MN[MC[i,0],0]-MN[MC[i,1],0]
    
    betam=MN[MC[i,0],1]-MN[MC[i,2],1]
    gammam=MN[MC[i,2],0]-MN[MC[i,0],0]
    
    B.append(np.array([[betai,0,betaj,0,betam,0],
                       [0,gammai,0,gammaj,0,gammam],
                       [gammai,betai,gammaj,betaj,gammam,betam]])*(1/(2*A)))
    

    k= t * abs(A) * (np.dot(np.transpose(B[i]), np.dot(D,B[i])))
    
    i,m,j = MC[i,:]    
    index=np.array([i*2,i*2+1,j*2,j*2+1,m*2,m*2+1])
    kglobal[np.ix_(index,index)]+=k


In [17]:
#CONDICIONES DE CONTORNO

#f[i,0]=1 se conoce la fuerza
#f[i,0]=0 no se conoce la fuerza
#f[i,1]=valor, su valor estara en la segunda columna.

f=np.zeros([2*n,2])

# introduzco a la matriz f las fuerzas conocidas

for i in range (nc):
    s=MNT[i,0]
    f[2*s,0]=1
    f[2*s,1]=MNT[i,1]
    f[2*s+1,0]=1
    f[2*s+1,1]=MNT[i,2]
    
# introduzco al vector f las fuerzas que son 0 porque no conozco su desplazamiento

for j in range (nb):
    if MNE[j,1]==1:
        s=MNE[j,0]
        f[2*s,0]=1
        f[2*s,1]=0
    if MNE[j,2]==1:
        s=MNE[j,0]
        f[2*s+1,0]=1
        f[2*s+1,1]=0
        
        
x=np.arange(n) 
x=np.delete(x,[np.concatenate((MNE[:,0].astype(int),MNT[:,0].astype(int)))]) #tiene adentro los nodos en los que no se pusieron condiciones
       

if nb+nc <n:
    f[2*x,0]=1
    f[2*x,1]=0
    f[2*x+1,0]=1
    f[2*x+1,1]=0


In [18]:
#El desplazamiento nulo a: Borde izquierdo (solo en x), Borde inferior (solo en y) ,Punto del medio de la chapa (en x e y)

indexd=np.sort( np.hstack((agujero1,agujero2,Smx,Smy)))
indexf = np.delete( np.arange(2*NumeroNodos) , indexd)



In [19]:
# selecciono las componentes de Kglobal y resuelvo [F]=[K'][u] como [K']^-1[F]=[u] 
#teniendo K' las componentes seleccionadas 
#segun las fuerzas conocidas desde un principio:

desplazamientos=np.dot(np.linalg.inv(kglobal[np.ix_(indexf,indexf)]),f[indexf,1])

#creo el vector desplazamientos, la notacion es igual que en el vector fuerzas:

d=np.zeros([2*NumeroNodos,2])
d[indexd,0]=1 
d[indexf,1]=desplazamientos
f[:,1]=np.dot(kglobal,d[:,1]) # fuerzas faltantes

In [20]:
#Calculo de las tensiones de los elementos

tension=[] 

for s in range (len(MC)) :
    i,m,j = MC[s,:]    
    index=np.array([i*2,i*2+1,j*2,j*2+1,m*2,m*2+1])
    valor=np.dot(np.dot(B[s],d[index,1]),D)
    tension.append(valor)
    
tension = np.array(tension)

In [21]:
#Calculo de las tensiones principales

tensionmax=[]
tensionmin=[]


for p in range (len(tension)):
    maximo=(tension[p,0]+tension[p,1])/2+np.sqrt(((tension[p,0]-tension[p,1])/2)**2+tension[p,2]**2)
    minimo=(tension[p,0]+tension[p,1])/2-np.sqrt(((tension[p,0]-tension[p,1])/2)**2+tension[p,2]**2)

    tensionmax.append(maximo)
    tensionmin.append(minimo)
    

tensionmax = np.array(tensionmax)
tensionmin = np.array(tensionmin)

#Criterio Tresca, promediotens> (tension de fluencia /2) , entra en fluencia

promediotens=(tensionmax-tensionmin)/2


In [22]:
Despx = d[:-1:2,1].reshape(-1,1)
Despy = d[1::2,1].reshape(-1,1)

In [23]:
DESP=np.hstack((Despx,Despy,np.zeros([NumeroNodos,1])))

In [24]:
desps = gmsh.view.add("Desplazamientos")

In [25]:
Desps = gmsh.view.addModelData(desps, 0, 'Agujero', 'NodeData', NodeInfo[0], DESP, numComponents=3)

In [26]:
Tensiones=gmsh.view.add('Tensiones promedio')

In [27]:
Tensiones = gmsh.view.addModelData(Tensiones, 0, 'Agujero', 'ElementData', ETAGS, promediotens.reshape(-1,1) , numComponents=1 )

In [28]:
print('La cantidad de elementos es',len(MC))
print('La cantidad de nodos es',n)

La cantidad de elementos es 60
La cantidad de nodos es 42


In [29]:
gmsh.fltk.run()

In [30]:
gmsh.finalize()